In [1]:
import os
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
import glob

/opt/anaconda3/envs/rag_projects_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_NAME = "gpt-4o-mini"

In [50]:
# Load the api key from .env file
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [51]:
# Context information about employees for the agent

context = {}
employees = glob.glob("knowledge-base/employees/*")

for employee in employees:
    name = employee.split(' ')[-1][:-3] # Take the last name of the employee
    doc = ""
    with open(employee, 'r') as f:
        doc = f.read()
    context[name] = doc



In [52]:
print(context["Harper"])

# HR Record

# Alex Harper

## Summary
- **Date of Birth**: March 15, 1993  
- **Job Title**: Sales Development Representative (SDR)  
- **Location**: Denver, Colorado  

## Insurellm Career Progression
- **July 2021**: Joined Insurellm as a Sales Development Representative, focusing on lead generation and nurturing B2B relationships.  
- **January 2022**: Promoted to Senior Sales Development Representative due to exceptional performance in converting leads into clients.  
- **October 2022**: Completed an Internal Leadership Training Program, enhancing skills in team collaboration and strategic selling. Currently mentoring junior SDRs.  
- **April 2023**: Became involved in a cross-departmental project to streamline the customer onboarding process, showcasing initiative and leadership.  

## Annual Performance History  
- **2021**:  
  - **Performance Rating**: 4.5/5  
  - **Key Achievements**: Exceeded lead generation targets by 30%. Introduced a new CRM analytics tool resulting in im

In [53]:
context.keys()

dict_keys(['Chen', 'Spencer', 'Tran', 'Blake', 'Lancaster', 'Thompson', 'Greene', 'Thomson', 'Trenton', 'Harper', 'Bishop', 'Carter'])

In [62]:
# Context information about products for the agent

products = glob.glob("knowledge-base/products/*")

for product in products:
    name = product.split(os.sep)[-1][:-3] # Take the last name of the employee
    print("Name:", name)
    doc = ""
    with open(product, 'r') as f:
        doc = f.read()
    context[name] = doc

Name: Rellm
Name: Markellm
Name: Homellm
Name: Carllm


In [63]:
print(context['Carllm'])


# Product Summary

# Carllm

## Summary

Carllm is an innovative auto insurance product developed by Insurellm, designed to streamline the way insurance companies offer coverage to their customers. Powered by cutting-edge artificial intelligence, Carllm utilizes advanced algorithms to deliver personalized auto insurance solutions, ensuring optimal coverage while minimizing costs. With a robust infrastructure that supports both B2B and B2C customers, Carllm redefines the auto insurance landscape and empowers insurance providers to enhance customer satisfaction and retention.

## Features

- **AI-Powered Risk Assessment**: Carllm leverages artificial intelligence to analyze driver behavior, vehicle conditions, and historical claims data. This enables insurers to make informed decisions and set competitive premiums that reflect true risk profiles.

- **Instant Quoting**: With Carllm, insurance companies can offer near-instant quotes to customers, enhancing the customer experience. The AI 

In [64]:
# Set the system message
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."


In [65]:
print(context.keys())

dict_keys(['Chen', 'Spencer', 'Tran', 'Blake', 'Lancaster', 'Thompson', 'Greene', 'Thomson', 'Trenton', 'Harper', 'Bishop', 'Carter', 'Rellm', 'Markellm', 'Homellm', 'Carllm'])


In [73]:
# Function to retrieve relveant context based on the found word

def get_relevant_context(msg):
    relevant_context = []
    for context_title, context_text in context.items():
        # print(context_title)x
        if context_title.lower() in msg.lower():
            relevant_context.append(context_text)
    return relevant_context

In [74]:
print(get_relevant_context("What is Rellm"))

["# Product Summary\n\n# Rellm: AI-Powered Enterprise Reinsurance Solution\n\n## Summary\n\nRellm is an innovative enterprise reinsurance product developed by Insurellm, designed to transform the way reinsurance companies operate. Harnessing the power of artificial intelligence, Rellm offers an advanced platform that redefines risk management, enhances decision-making processes, and optimizes operational efficiencies within the reinsurance industry. With seamless integrations and robust analytics, Rellm enables insurers to proactively manage their portfolios and respond to market dynamics with agility.\n\n## Features\n\n### AI-Driven Analytics\nRellm utilizes cutting-edge AI algorithms to provide predictive insights into risk exposures, enabling users to forecast trends and make informed decisions. Its real-time data analysis empowers reinsurance professionals with actionable intelligence.\n\n### Seamless Integrations\nRellm's architecture is designed for effortless integration with ex

In [75]:
# Function to add the context to the message to send to the LLM

def add_context(msg):
    relevant_context = get_relevant_context(msg)
    if relevant_context:
        msg += "\n\nThe following additional context might be releant to the question: \n\n"
        for relvant_msg in relevant_context:
            msg += relvant_msg + "\n\n"
    return msg

In [76]:
print(add_context("What is Rellm?"))

What is Rellm?

The following additional context might be releant to the question: 

# Product Summary

# Rellm: AI-Powered Enterprise Reinsurance Solution

## Summary

Rellm is an innovative enterprise reinsurance product developed by Insurellm, designed to transform the way reinsurance companies operate. Harnessing the power of artificial intelligence, Rellm offers an advanced platform that redefines risk management, enhances decision-making processes, and optimizes operational efficiencies within the reinsurance industry. With seamless integrations and robust analytics, Rellm enables insurers to proactively manage their portfolios and respond to market dynamics with agility.

## Features

### AI-Driven Analytics
Rellm utilizes cutting-edge AI algorithms to provide predictive insights into risk exposures, enabling users to forecast trends and make informed decisions. Its real-time data analysis empowers reinsurance professionals with actionable intelligence.

### Seamless Integration

In [77]:
# Chat function

def chat(msg, history):
    messages = [{"role":"system","content":system_message}]
    message = add_context(msg)
    messages.append({"role":"user", "content":message})

    stream = openai.chat.completions.create(model=MODEL_NAME, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [78]:
# Set up the Gradio Interface
view = gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
